## Purpose

The purpose of this file is to use the filtered fire data created in the data_acquisition script to generate a smoke estimate for Pahrump, NV on an annual and per-fire basis.

### Decisions and Assumptions in Creation of Smoke Estimate
To calculate smoke estimates we first considered which features of fires might be most relevent. We decided on type of fire, acres burned, distance to town, recency of other fires in the same area, and year of fire recording. We felt these were the most likely elements which were easily accessible in our dataset and they aligned with information from [U.S. Department of the Interior Pubilcations](https://www.doi.gov/wildlandfire/news/increasing-wildfires-are-causing-greater-air-pollution). If given more time, it would also be interesting to pull vegetation for burned areas (e.g., arid climates may have less to burn and thus less smoke) as well as weather conditions (e.g., windy conditions could disperse smoke more readily than stagnant conditions).

We had the following initial set of questions regarding our features:
   1. Do perscribed burns create as much smoke as wildfires?
   2. How is acres burned related to smoke production?
   3. How does distance to the edge of the fire relate to smoke dispursion?
   4. How much does burning an area reduce smoke if its burned again within 2 years?
   5. How reliable are estimates for burned area and thus smoke per year?

We completed the following research (numbering consistent with questions above) and determined the following mathematical relationships for each feature.
    
   1. Per David Frisbey's 2008 thesis "[A comparison of smoke emissions from prescribed burns and wildfires](https://scholarworks.sjsu.edu/cgi/viewcontent.cgi?referer=&httpsredir=1&article=4554&context=etd_theses)", "The results suggest that the smoke impacts of a wildfire may not be any greater than a prescribed burn when compared using the methodology. This research demonstrates how a combination of the fuel load and the size of the burn may be more significant in controlling downwind concentration of PM10 than the atmospheric conditions. Even when there is a planned burn under prescribed meteorological conditions there can be significant impacts if the size of the burn and fuel loading are not also considered." Referencing [Forest Service Professionals Prepare for a Prescribed Burn](https://www.fs.usda.gov/features/professionals-prepare-for-prescribed-burn), we can see that forestry services do take fuel moisture, forest stand characteristics, historical data, terrain, and elevation into account. Given David's findings, and assuming the Forest Service correctly accounts for the fuel variables to create less intense blazes, we will multiply the smoke estimate for perscribed burns by 0.50.  
    
   2. Composition of fuel certainly impacts the amount of smoke generated per acre burned. Given we are not currently bringing in additional vegetation information, we will assume a linear relationship between acres burned and smoke produced.
    
   3. Many factors impact smoke dispursion (direction of terrain, wind, other atmospheric conditions), but we will focus on distance to town as our primary variable. We will assume that smoke dispurses like light (1/distance^2) relationship, see [Nasa.gov](https://imagine.gsfc.nasa.gov/features/yba/M31_velocity/lightcurve/more.html#:~:text=Notice%20that%20as%20the%20distance,proportional%20to%20the%20distance%20squared) for more details.
    
   4. Fires which attempt to burn in areas burnt within 2 years should produce much less smoke than the original fires which burned there (see [Burn out: Frequent fires are changing Western landscapes](https://www.hcn.org/articles/climate-desk-wildfire-burn-out-frequent-fires-are-changing-western-landscapes#:~:text=However%2C%20overall%2C%20re%2Dburns,just%20fine%2C%E2%80%9D%20he%20said.) for more details). We will multiply smoke estimates for fires burning in burned areas by a factor of 0.20. 
    
   5. The USGS wildland fire metadata note that "Areas burned prior to 1984 in this dataset represent only a fraction of what actually burned. While areas burned on or after 1984 are much more accurate and complete, errors still can and do occur." Given the underestimation of acres burned, we will multiple acreage estimates for fires prior to 1984 by 1.5.
    
Using the above research, we arrive at a base smoke estimate of: (Acres burned)/(Distance to town)^2. If fires occurred prior to 1984 we will multiply this estimate by 1.5, if the fire was prescribed we will multiply the smoke estimator by 0.5, and if the area was burned in the previous 2 years we will multiply the smoke estimator by a factor of 0.2.

First we begin by loading common python libraries and the filtered fire data created in the data_acquisiton script.

In [6]:
#Imports
import os, json, time
import pandas as pd
import numpy as np

#Importing the filtered fire data for 1963 onward within 1250 miles of Pahrump, NV
fire_data = pd.read_csv('../intermediate_data/filtered_fire_info.csv')

#Checking the output
if fire_data['Fire_Year'].max() > 2023:
    print("ERROR - fires after 2023 included in data")
else:
    print("The max fire year is {0}".format(fire_data['Fire_Year'].max()))
if fire_data['Fire_Year'].min() < 1963:
    print("ERROR - fires before 1963 included in data")
else:
    print("The min fire year is {0}".format(fire_data['Fire_Year'].min()))
if fire_data['shortest_dist'].max() > 1250:
    print("ERROR - fires beyond 1250 miles included in data")
else:
    print("The fartherst fire is {0} miles from town".format(fire_data['shortest_dist'].max()))
    print("The closest fire is {0} miles from town".format(fire_data['shortest_dist'].min()))
print("There are {0} fires after 1963 within 1250 miles of Pahrump, NV.".format(len(fire_data)))

#Outputting example of what file looks like
print()
print("The filtered fire data looks like:")
print(fire_data.head())

The max fire year is 2020
The min fire year is 1963
The fartherst fire is 1249.99 miles from town
The closest fire is 8.52 miles from town
There are 81351 fires after 1963 within 1250 miles of Pahrump, NV.

The filtered fire data looks like:
   OBJECTID Assigned_Fire_Type  Fire_Year     GIS_Acres  \
0     14299           Wildfire       1963  40992.458271   
1     14300           Wildfire       1963  25757.090203   
2     14301           Wildfire       1963  45527.210986   
3     14302           Wildfire       1963  10395.010334   
4     14303           Wildfire       1963   9983.605738   

  Overlap_Within_1_or_2_Flag  shortest_dist  
0                        NaN         549.11  
1                        NaN         569.27  
2                        NaN         556.75  
3                        NaN         479.44  
4                        NaN         541.90  
